# urllib

## Preliminaries

Check that we are running the expected version of Python (should be something like 3.5.3).

In [ ]:
import sys
sys.version

Load some functions from `urllib`.

In [ ]:
from urllib.parse import urlencode, urljoin
from urllib.request import urlopen, urlretrieve, HTTPError, URLError
from urllib.robotparser import RobotFileParser

Other packages.

In [ ]:
import os

## Working with URLs

### URL Encoding

Some characters (like commas and spaces) need to be specially encoded when they are embedded in URLs. You can try this out [here](https://www.urlencoder.org/).

The `urlencode()` function accepts a dictionary then encodes and concatenates each of its elements.

In [ ]:
params = {
    'q': 'web scraping',
    'ie': 'UTF-8'
}
urlencode(params)

### Builing a URL

With `urljoin()` you can compose an absolute URL and a relative URL.

In [ ]:
urljoin('https://www.google.com', '/search')

Now joining and encoding to form the complete URL for a Google query.

In [ ]:
urljoin('https://www.google.com', '/search')+'?'+urlencode(params)

## Retrieving a Web Page

URLs on the [Private Property](https://www.privateproperty.co.za/) yield a hierarchy of information. Consider the following:

`https://www.privateproperty.co.za/for-sale/kwazulu-natal/drakensberg/richmond-and-surrounds/byrne-valley/T479322`

It tells us that the property is

- "for sale" (as opposed to "to rent");
- in the province of KwaZulu-Natal;
- in the Drakensberg area;
- in the vicinity of Richmond;
- more specifically, the Byrne Valley; and
- has a *unique* ID of T479322.

It's the last bit that's really useful because it allows us to navigate directly to this property from the Home Page.

For brevity we'll use [bitly](https://bitly.com/) to generate a short URL.

In [ ]:
URL = "http://bit.ly/2eL160q"

Retrieve the HTTP response.

In [ ]:
html = urlopen(URL)

In [ ]:
html

Take a look at the contents.

In [ ]:
html.read()

## More Robustly Retrieving the Page

Things can go wrong when you try to retrieve a web page. We have all had this experience in a browser. The same thing can happen in code.

Here are some possibilities:

- 404 error (page not found);
- 500 error (server error); or
- DNS failure or server not found.

Write code to cater for these cases.

In [ ]:
# Try these URLs:
#
# URL = "https://www.privateproperty.co.za/for-sale/kwazulu-natal/durban/"
# URL = "http://www.totally-bogus-server.com/"

try:
	html = urlopen(URL)
except HTTPError as e:
	print(e)
    #
    # [Code to handle this error.]
except URLError as e:
	print(e)
    #
    # [Code to handle this error.]
else:
    print("Sucessfully retrieved the HTML for %s." % URL)
    #
    # This branch is not really necessary.

We've been using `urlopen()` which returns the contents of a URL as a string. What if we want to actually store a local copy of the URL? This can be handy for offline processing or if we want to archive a version of the page.

## Retrieving to a File

To download a URL to a local file use `urlretrieve()`.

In [ ]:
filepath, http_message = urlretrieve(URL)

The return value is a tuple containing

- the path of the file in which the page is stored and
- a `HTTPMessage` object.

By default the file is downloaded to a temporary location.

In [ ]:
filepath

Check on the resulting file.

In [ ]:
os.stat(filepath)

The `HTTPMessage` object contains the response headers and behaves like a dictionary.

In [ ]:
http_message.as_string()

In [ ]:
http_message.items()

In [ ]:
http_message['Date']

We can also save the file to a specified location.

In [ ]:
urlretrieve(URL, filename = 'T479322.html')

## Parsing `robots.txt`

Create an object for parsing `robots.txt`.

In [ ]:
robots = RobotFileParser()

Let's work with the `robots.txt` from [PrivateProperty](https://www.privateproperty.co.za/robots.txt).

In [ ]:
robots.set_url('https://www.privateproperty.co.za/robots.txt')
robots.read()

The `can_fetch()` method accepts two arguments:

- a User Agent and
- a URL.

In [ ]:
robots.can_fetch('*', 'https://www.privateproperty.co.za/Portal/Home')

This URL is not allowed for anybody...

In [ ]:
robots.can_fetch('*', 'https://www.privateproperty.co.za/Login')

... except Google, who is welcome.

In [ ]:
robots.can_fetch('Mediapartners-Google', 'https://www.privateproperty.co.za/Login')

## Cleaning Up

In [ ]:
os.remove('T479322.html')